In [0]:
# Source files
blob_account_name = "blob_account_name"
blob_container_name = "blob_container_name"
blob_relative_path = "blob_relative_path"
blob_sas_token = dbutils.secrets.get("sas_token_name", "sastoken")
csv_file ="csv_file.csv"

In [0]:
wasbs_path = f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/{blob_relative_path}/{csv_file}"
spark.conf.set(f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net", blob_sas_token)

In [0]:
# Table & View
raw_miscvendor = "raw_miscvendor"
miscvendor_view = "miscvendor_view"
miscvendor_data = "miscvendor_data"

In [0]:
# Cache & Create Raw Data from Source files
spark.read.csv(wasbs_path, header='true').createOrReplaceTempView(miscvendor_view)
spark.sql(f"""
    CREATE OR REPLACE TABLE {raw_miscvendor}
    SELECT Index, Vendor_Name, Quote, PO, Quantity, Description, Unit_Price, Start_Date, End_Date, Months, Days, Contract_Unit, Total_Cost, Monthly_Cost, Monthly_Cost_USD, Daily_Cost_USD, sc_system, sc_component, sc_costowner, sc_env, sc_type, sc_region, sc_createdby, sc_provider
    FROM {miscvendor_view}
""")

In [0]:
# Overwrite data

spark.sql(f"""
CREATE OR REPLACE TABLE {miscvendor_data}
(
    SELECT 
    --System fields
    Vendor_Name AS Vendor, 
    explode(sequence(Start_Date::DATE, End_Date::DATE, interval 1 day)) AS Date, 
    Daily_Cost_USD::DOUBLE AS AllocationCost,
    CASE WHEN Vendor_Name = "Auth0" THEN "Identity" ELSE NULL END AS System,
    --Tag fields
    sc_system, sc_component, sc_env, sc_region, sc_type, sc_costowner, sc_createdby, sc_provider
    FROM {miscvendor_view}
    WHERE Vendor_Name != "Elastic"
)
""")

In [0]:
dbutils.notebook.exit("OK")